## Header Files

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from datetime import datetime
import seaborn as sns
from itertools import combinations
import math
import random
from scipy.stats import skew, kurtosis
import warnings
from PowerFlowAnalysis import PowerFlow
import time
from statistics import mean


# Suppress DeprecationWarning
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Feature Engineering

In [6]:
data_file_path = 'Sample_Data8.xlsx' #INPUT
dist_file_path = 'Distance2.xlsx' #READY
ratings_file_path = 'Ratings.xlsx' #READY
line_data_path = 'Linedata2.csv' #READY
distdf = pd.read_excel(dist_file_path)
ratings_df = pd.read_excel(ratings_file_path)
line_data_df = pd.read_csv(line_data_path)
df = pd.read_excel(data_file_path)
df

,Period (Hr),Prosumer_Id,Solar,Storage,Total (Units),Load Forecast (Units),Unit Price,Profit_pref,Rating_pref,Solar Panel Rating,DSM
0,1,2,0.1000,0.800000,0.900000,0.5755,9,0.3,0.7,2,0.894151
1,1,3,0.1050,0.200000,0.305000,0.4240,8,0.8,0.2,2,0.984550
2,1,4,0.0000,0.000000,0.000000,0.8620,10,0.7,0.3,3,0.734713
3,1,5,0.1800,0.450000,0.630000,0.5740,9,0.2,0.8,3,0.598530
4,1,6,0.1875,0.500000,0.687500,0.6020,9,0.2,0.8,3,0.158909
...,...,...,...,...,...,...,...,...,...,...,...
283,32,6,0.1875,0.037961,0.225461,0.2590,7,0.5,0.5,3,1.000000
284,32,7,0.0000,0.000000,0.000000,0.3770,7,0.6,0.4,5,0.000000
285,32,8,0.1875,0.078192,0.265692,0.2590,7,0.1,0.9,2,1.000000
286,32,9,0.6250,0.117808,0.742808,0.3720,7,0.2,0.8,10,1.000000


In [7]:
df['Value'] = round(df['Total (Units)'] - df['Load Forecast (Units)'],3)
df['Category'] = np.where(df['Value'] < 0, 'B', np.where(df['Value'] > 0, 'S', 'B'))
df['Value'] = np.where(df['Value'] > 0, df['Value'], -1 * df['Value'])
df['Profit_pref']=round(df['Profit_pref'],1)
df['Rating_pref']=round(df['Rating_pref'],1)
df['DSM'] = round(df['DSM'],0)
df.to_csv('df.csv', index=False)
#df

## Creating Bus

In [8]:
def create_bus_data(period_data):
    
#                   0   1   2   3    4    5   6   7  8     9
#                 |Bus|Type|Vp|theta|PGi|QGi|PLi|QLi|Qmin|Qmax|
    
    bus_data = []
    
    bus_data.append({'Bus': 1, 'Type': 1, 'Vp': 1, 'theta': 0, 'PGi': 0, 'QGi': 0, 'PLi': 0, 'QLi': 0, 'Qmin': 0, 'Qmax': 0})

    for index, row in period_data.iterrows():
        if row['Category'] == 'B':
            bus_data.append({'Bus': row['Prosumer_Id'], 'Type': 3, 'Vp': 1, 'theta': 0.0, 
                             'PGi': 0, 'QGi': 0, 'PLi': row['Value']*1000*4, 'QLi': 0, 'Qmin': 0, 'Qmax': 0})
        elif row['Category'] == 'S':
            bus_data.append({'Bus': row['Prosumer_Id'], 'Type': 3, 'Vp': 1, 'theta': 0.0, 
                             'PGi': 0, 'QGi': 0, 'PLi': 0, 'QLi': 0, 'Qmin': 0, 'Qmax': 0})
            
    bus_df = pd.DataFrame(bus_data)
    
    return bus_df

In [9]:
linedata = line_data_df.to_numpy()
BMva = 1e5  # Base VA

## SDR

In [10]:
mini_price = 3.6

In [11]:
sdr_df = pd.DataFrame(columns=['Period', 'SDR'])

for period in df['Period (Hr)'].unique():
    data = df[df['Period (Hr)'] == period]

    TBP = data[data['Category'] == 'B']['Value'].sum()
    TSP = data[data['Category'] == 'S']['Value'].sum()

    sdr = TSP/TBP
    sdr_df = pd.concat([sdr_df, pd.DataFrame({'Period': [period], 'SDR': [sdr]})], ignore_index=True)

sdr_df

C:\Users\sdash\AppData\Local\Temp\ipykernel_11904\596540504.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sdr_df = pd.concat([sdr_df, pd.DataFrame({'Period': [period], 'SDR': [sdr]})], ignore_index=True)


,Period,SDR
0,1,0.664852
1,2,0.575980
2,3,1.030671
3,4,1.000000
4,5,1.067069
5,6,1.405251
6,7,0.931452
7,8,0.998324
8,9,1.136822
9,10,1.035054


## Floyd-Warshall (Distance Based)

In [12]:
distdf

,ID,1,2,3,4,5,6,7,8,9,10
0,1,0,2,0,0,0,0,0,0,0,0
1,2,2,0,8,12,0,16,0,0,0,0
2,3,0,8,0,0,0,0,0,0,0,22
3,4,0,12,0,0,10,0,0,0,0,0
4,5,0,0,0,10,0,0,0,0,0,4
5,6,0,16,0,0,0,0,14,0,0,0
6,7,0,0,0,0,0,14,0,18,20,0
7,8,0,0,0,0,0,0,18,0,0,6
8,9,0,0,0,0,0,0,20,0,0,0
9,10,0,0,22,0,4,0,0,6,0,0


In [13]:
transaction_df = pd.DataFrame(columns=['Period', 'Seller', 'Buyer', 'Units', 'Unit Price','Savings','Loss'])

In [14]:
def floyd_warshall(graph):
    num_nodes = len(graph)
    # Replace all 0 values with infinity
    graph = np.where(graph == 0, np.inf, graph)
    distance_matrix = np.array(graph, dtype=float)
    intermediate_nodes = np.zeros((num_nodes, num_nodes), dtype=int)

    # Initialize intermediate nodes matrix
    for i in range(num_nodes):
        for j in range(num_nodes):
            intermediate_nodes[i, j] = j if i != j and graph[i, j] != np.inf else -1

    # Floyd-Warshall algorithm with intermediate nodes tracking
    for k in range(num_nodes):
        for i in range(num_nodes):
            for j in range(num_nodes):
                if distance_matrix[i, j] > distance_matrix[i, k] + distance_matrix[k, j]:
                    distance_matrix[i, j] = distance_matrix[i, k] + distance_matrix[k, j]
                    intermediate_nodes[i, j] = intermediate_nodes[i, k]

    np.fill_diagonal(distance_matrix, np.inf)

    return distance_matrix, intermediate_nodes

distdf.set_index('ID', inplace=True)


distance_matrix, intermediate_nodes = floyd_warshall(distdf.values)
short_dist_df = pd.DataFrame(distance_matrix, columns=distdf.columns, index=distdf.index)

print("Shortest Distance Matrix:")
short_dist_df

Shortest Distance Matrix:


,1,2,3,4,5,6,7,8,9,10
ID,,,,,,,,,,
1,inf,2.0,10.0,14.0,24.0,18.0,32.0,34.0,52.0,28.0
2,2.0,inf,8.0,12.0,22.0,16.0,30.0,32.0,50.0,26.0
3,10.0,8.0,inf,20.0,26.0,24.0,38.0,28.0,58.0,22.0
4,14.0,12.0,20.0,inf,10.0,28.0,38.0,20.0,58.0,14.0
5,24.0,22.0,26.0,10.0,inf,38.0,28.0,10.0,48.0,4.0
6,18.0,16.0,24.0,28.0,38.0,inf,14.0,32.0,34.0,38.0
7,32.0,30.0,38.0,38.0,28.0,14.0,inf,18.0,20.0,24.0
8,34.0,32.0,28.0,20.0,10.0,32.0,18.0,inf,38.0,6.0
9,52.0,50.0,58.0,58.0,48.0,34.0,20.0,38.0,inf,44.0


In [15]:
start_node = 1
end_node = 9
route = [str(start_node)] 
current_node = start_node

while current_node != end_node:
    next_node = intermediate_nodes[current_node-1, end_node-1] 
    if next_node == -1:
        print(f"No route from {start_node} to {end_node}")
        break
    route.append(str(next_node + 1)) 
    current_node = next_node + 1 

print(f"Shortest Route from {start_node} to {end_node}: {' -> '.join(route)}")

Shortest Route from 1 to 9: 1 -> 2 -> 6 -> 7 -> 9


# Algorithm

In [16]:
iter_sharpe = []

In [17]:
data = []
columns = ['Period', 'V', 'Del', 'Lij', 'Spij', 'Pij', 'Qij', 'Pg', 'Qg', 'Pi','Qi','Iij','Pl']

In [18]:
def calculate_sdr_price(sdr, buyer_UBP):
    maxi_price = buyer_UBP
    mid_value = (maxi_price + mini_price) / 2
    b = 1 / maxi_price
    a = 1 / mid_value - b
    return round(1 / (a * sdr + b), 2)

def calculate_trading_price(sdr_price, rating_buyer, rating_seller, buyer_rating_pref, buyer_profit_pref, seller_rating_pref, seller_profit_pref, trading_units, buyer_UBP,p):
    price_buyer = sdr_price*(1 +  p[0] * (rating_seller * buyer_rating_pref -  buyer_profit_pref)) 
    price_seller = sdr_price*(1 - p[0] * (rating_buyer * seller_rating_pref - seller_profit_pref))
    trading_price = round((price_buyer + price_seller) / 2, 2)
    trading_price = (mini_price + 0.01) if trading_price < mini_price else trading_price
    trading_price = (buyer_UBP - 0.01) if trading_price > buyer_UBP else trading_price
    return trading_price

def calculate_utilities(trading_unit, buyer_UBP, trading_price, rating_buyer, rating_seller, buyer_rating_pref,
                        buyer_profit_pref, seller_rating_pref, seller_profit_pref, seller_dsm, buyer,seller, dist_factor, bpm,spm):
    buyer_profit = round(trading_unit * (buyer_UBP - trading_price),3)
    seller_profit = round(trading_unit * (trading_price - mini_price),3)
    
    m = (bpm+spm)/2
    
    buyer_util_profit = round(buyer_profit_pref * (buyer_UBP - trading_price), 3)
    buyer_util_rating = round(m * buyer_rating_pref * rating_seller, 3)
    
    seller_util_profit = round(seller_profit_pref * (trading_price - mini_price), 3)
    seller_util_rating = round(m * rating_buyer * seller_rating_pref , 3)
    
    dist_factor = round(dist_factor.loc[buyer,seller],3)
    buyer_utility = round(buyer_util_profit + buyer_util_rating -  dist_factor,3)
    seller_utility = round(seller_util_profit + seller_util_rating -  dist_factor,3)
    
    return buyer_utility, seller_utility, buyer_profit, seller_profit

def create_transaction_df(period, seller, buyer, sdr_price, trading_price, trading_units, seller_profit,
                          buyer_profit, seller_utility, buyer_utility, max_utility):
    return pd.DataFrame({
        'Period': [period],
        'Seller': [seller],
        'Buyer': [buyer],
        'SDR Price': [sdr_price],
        'Trading Price': [trading_price],
        'Trading Energy': [trading_units],
        'Seller Profit': [seller_profit],
        'Buyer Profit': [buyer_profit],
        'Seller Utility': [seller_utility],
        'Buyer Utility': [buyer_utility],
        'Total_Utility': [max_utility]
    })
import pandas as pd

def DistanceFactor(period_data):
    buyers = period_data[period_data['Category'] == 'B']
    sellers = period_data[period_data['Category'] == 'S']

    distance_factors = pd.DataFrame(index=buyers['Prosumer_Id'], columns=sellers['Prosumer_Id'])

    total_shortest_distance = {}
    for seller in sellers['Prosumer_Id']:
        total_shortest_distance[seller] = 0
        for buyer in buyers['Prosumer_Id']:
            total_shortest_distance[seller] += short_dist_df.loc[buyer, seller]

    for buyer in buyers['Prosumer_Id']:
        for seller in sellers['Prosumer_Id']:
            shortest_cumulative_distance = short_dist_df.loc[buyer, seller]
            distance_factors.loc[buyer, seller] = shortest_cumulative_distance / total_shortest_distance[seller]

    return distance_factors


In [19]:
def calculate_utility_matrix_for_period(df, ratings_df, rejected, period,p,bpm, spm):
    total_utilities_df = pd.DataFrame(columns=['Period', 'Seller', 'Buyer', 'Buyer Utility',
                                               'Seller Utility', 'Trading price', 'Trading unit', 'UBP',
                                               'Total_Utility', 'Energy Loss'])
    
    period_data = df[df['Period (Hr)'] == period]
    sdr = sdr_df.loc[sdr_df['Period'] == period, 'SDR'].values[0]
    buyer_data = period_data[period_data['Category'] == 'B']
    seller_data = period_data[period_data['Category'] == 'S']
    distance_factor = DistanceFactor(period_data)
    
    buyer_profit_list = []
    seller_profit_list = []

    for _, row_buyer in buyer_data.iterrows():
        buyer = row_buyer['Prosumer_Id']
        buyer_UBP = row_buyer['Unit Price']
        sdr_price = calculate_sdr_price(sdr, buyer_UBP)
        
        for _, row_seller in seller_data.iterrows():
            seller = row_seller['Prosumer_Id']
            rating_buyer = ratings_df.loc[ratings_df['ID'] == buyer, 'Ratings'].values[0]
            rating_seller = ratings_df.loc[ratings_df['ID'] == seller, 'Ratings_avg'].values[0]
            buyer_profit_pref = buyer_data.loc[buyer_data['Prosumer_Id'] == buyer, 'Profit_pref'].values[0]
            buyer_rating_pref = buyer_data.loc[buyer_data['Prosumer_Id'] == buyer, 'Rating_pref'].values[0]
            seller_rating_pref = seller_data.loc[seller_data['Prosumer_Id'] == seller, 'Rating_pref'].values[0]
            seller_profit_pref = seller_data.loc[seller_data['Prosumer_Id'] == seller, 'Profit_pref'].values[0]
            seller_dsm = seller_data.loc[seller_data['Prosumer_Id'] == seller, 'DSM'].values[0]

            trading_unit = round(min(row_buyer['Value'], row_seller['Value']), 3)
            trading_price = calculate_trading_price(sdr_price, rating_buyer, rating_seller, buyer_rating_pref,
                                                    buyer_profit_pref, seller_rating_pref, seller_profit_pref, trading_unit, buyer_UBP,p)
            
            
            buyer_utility, seller_utility, buyer_profit, seller_profit = calculate_utilities(trading_unit, buyer_UBP, trading_price, rating_buyer, rating_seller, buyer_rating_pref,
                        buyer_profit_pref, seller_rating_pref, seller_profit_pref, seller_dsm, buyer, seller, distance_factor,bpm, spm)
            
            if (trading_unit == 0) or ((buyer,seller) in rejected):
                buyer_utility = 0
                seller_utility = 0
                
            buyer_profit_list.append(buyer_profit/trading_unit*buyer_profit_pref)
            seller_profit_list.append(seller_profit/trading_unit*seller_profit_pref)
            
            total_utility = round(buyer_utility * seller_utility, 6)
            

            new_row = create_transaction_df(period, seller, buyer, sdr_price, trading_price, trading_unit,
                                            seller_profit, buyer_profit, seller_utility, buyer_utility, total_utility)
                
            total_utilities_df = pd.concat([total_utilities_df, new_row], ignore_index=True)

    utility_matrix = total_utilities_df.pivot_table(index='Seller', columns='Buyer', values='Total_Utility', fill_value=0)
    return utility_matrix, total_utilities_df, mean(buyer_profit_list), mean(seller_profit_list)

def make_transactions_with_priority(df, ratings_df, price_df):
    total_utilities_list = []

    for period in df['Period (Hr)'].unique():
        
        p= [0.4]
        
        limits = np.array([
       [0, 11000, 4000, 3000, 2000, 1000, 1000, 800, 800, 800], #0
        [5000, 0, 4000, 4000, 2000, 3500, 1000, 1000, 800, 500], #1
        [4000, 22, 0, 3000, 2000, 1000, 1000, 900, 800, 1800], #2
        [31, 32, 33, 0, 2500, 1000, 1000, 1000, 800, 800], #3
        [41, 42, 0, 44, 0, 1000, 1000, 1000, 800, 2500], #4
        [51, 52, 0, 54, 55, 0, 2000, 1000, 800, 800], #5
        [61, 62, 63, 0, 65, 66,0, 1500, 2500, 1000], #6
        [71, 72, 73, 74, 75, 76, 77, 0, 800, 2500], #7
        [81, 82, 83, 84, 85, 86, 87, 88, 0,  1000], #8
        [91, 92, 93, 94, 95, 96, 97, 98, 99, 0]
        ])
        rejected = []
        busdata = create_bus_data(df[df['Period (Hr)']==period])
        
        sdr = sdr_df.loc[sdr_df['Period'] == period, 'SDR'].values[0]
        utility_matrix, total_utilities_df, bpm, spm = calculate_utility_matrix_for_period(df, ratings_df, rejected, period,p,3,3)
        utility_matrix, total_utilities_df, _b, _s = calculate_utility_matrix_for_period(df, ratings_df, rejected, period,p,bpm,spm)
        period_data = df[df['Period (Hr)'] == period].copy()
        buyer_data = period_data[period_data['Category'] == 'B']
        
        seller_data = period_data[period_data['Category'] == 'S']
        distance_factor = DistanceFactor(period_data)
        
        #TRANSACTION
        
        while True:
            max_utility = utility_matrix.values.max()
            if max_utility <= 0:
                break
                
            seller, buyer = np.unravel_index(np.argmax(utility_matrix.values), utility_matrix.shape)
            seller, buyer = utility_matrix.index[seller], utility_matrix.columns[buyer]

            buyer_UBP = buyer_data.loc[buyer_data['Prosumer_Id'] == buyer, 'Unit Price'].values[0]
            sdr_price = calculate_sdr_price(sdr, buyer_UBP)
                
                
            rating_buyer = ratings_df.loc[ratings_df['ID'] == buyer, 'Ratings'].values[0]
            rating_seller = ratings_df.loc[ratings_df['ID'] == seller, 'Ratings_avg'].values[0]
            buyer_profit_pref = buyer_data.loc[buyer_data['Prosumer_Id'] == buyer, 'Profit_pref'].values[0]
            buyer_rating_pref = buyer_data.loc[buyer_data['Prosumer_Id'] == buyer, 'Rating_pref'].values[0]
            seller_rating_pref = seller_data.loc[seller_data['Prosumer_Id'] == seller, 'Rating_pref'].values[0]
            seller_profit_pref = seller_data.loc[seller_data['Prosumer_Id'] == seller, 'Profit_pref'].values[0]
            seller_dsm = seller_data.loc[seller_data['Prosumer_Id'] == seller, 'DSM'].values[0]
            
            row_buyer = period_data[period_data['Prosumer_Id'] == buyer]
            row_seller = period_data[period_data['Prosumer_Id'] == seller]
            
            trading_units = round(min(row_buyer['Value'].values[0], row_seller['Value'].values[0]), 3)
            trading_price = calculate_trading_price(sdr_price, rating_buyer, rating_seller, buyer_rating_pref,
                                                    buyer_profit_pref, seller_rating_pref, seller_profit_pref, trading_units, buyer_UBP,p)
        
            buyer_utility, seller_utility, buyer_profit, seller_profit = calculate_utilities(trading_units, buyer_UBP, trading_price, rating_buyer, rating_seller, buyer_rating_pref,
                        buyer_profit_pref, seller_rating_pref, seller_profit_pref, seller_dsm, buyer, seller, distance_factor,bpm,spm)
            
            if (trading_units == 0) or ((buyer,seller) in rejected):
                buyer_utility = 0
                seller_utility = 0

            total_utility = round(buyer_utility * seller_utility, 6)
            
            if (buyer,seller) in rejected:
                continue
                
            if trading_units <= 0:
                utility_matrix[buyer] = 0
                utility_matrix.loc[seller] = 0
                continue
                
            
            period_data.loc[period_data['Prosumer_Id'] == seller, 'Value'] -= trading_units
            period_data.loc[period_data['Prosumer_Id'] == buyer, 'Value'] -= trading_units
            
            
            #POWER FLOW
            busdata.loc[busdata['Bus']==seller, 'Type'] = 2
            busdata.loc[busdata['Bus']==seller, 'PGi'] += (trading_units*1000*4)
            busdata_array = busdata.to_numpy()
            power_flow = PowerFlow(busdata_array, linedata) 
            V, Del, Iter, check = power_flow.loadflow(busdata_array[:, 2], busdata_array[:, 3], BMva)
            Lij,Sij,Pij,Qij,Pg,Qg, Pi, Qi, Iij, Pl = power_flow.powerflow(V, np.deg2rad(Del), BMva)
            flag = False
            
            #LIMITS CHECKING
            for i in range(10):
                for j in range(10):
                    if( (i<j) and (limits[i][j] - max(np.abs(Pij[i][j]),np.abs(Pij[j][i])) < 0) or (check == False)):
                        rejected.append((buyer,seller))
                        utility_matrix.loc[seller, buyer] = 0
                        period_data.loc[period_data['Prosumer_Id'] == seller, 'Value'] += trading_units
                        period_data.loc[period_data['Prosumer_Id'] == buyer, 'Value'] += trading_units
                        
                        busdata.loc[busdata['Bus']==seller, 'PGi'] -= (trading_units*1000*4)
                        if busdata.loc[busdata['Bus'] == seller, 'PGi'].values[0] == 0:
                            busdata.loc[busdata['Bus'] == seller, 'Type'] = 3
                            
                        flag = True
                        
                        print(f"Period: {period}, Rejecting Transaction: {buyer}, {seller}, Line Limit: {limits[i][j]}, Line: {i} -> {j}, Current Power Flow ({i}->{j}): {np.abs(Pij[i][j])},  Total Utility {max_utility},  Trading Units {trading_units} ")
                        break
                        
                if flag:
                    break
                        
            if flag:
                continue
                        
            print(f"Period: {period}, Transaction Approved: {buyer}, {seller}")

            
            data.append([period, V, Del, Lij, Sij, Pij, Qij, Pg, Qg, Pi, Qi, Iij, Pl])
            
            if (period_data.loc[period_data['Prosumer_Id'] == seller, 'Value'].values[0] == 0):
                utility_matrix.loc[seller] = 0
            else:
                utility_matrix[buyer] = 0
            
            total_utilities_list.append(create_transaction_df(period, seller, buyer, sdr_price, trading_price,
                                                              trading_units, seller_profit, buyer_profit, seller_utility,
                                                              buyer_utility, max_utility))

            print(f"Transaction: Period {period}, Buyer: {buyer}, Seller: {seller}, Total Utility {max_utility}, Trading Units {trading_units}, Price {trading_price}")

            
            
        # SURPLUS TO GRID
        seller_data = period_data[period_data['Category'] == 'S']
        for _, row_seller in seller_data.iterrows():
            
            if round(row_seller['Value'],3)>0:
                
                seller = row_seller['Prosumer_Id']
                trading_units = round(row_seller['Value'],3)
                
                
                #POWER FLOW
                busdata.loc[busdata['Bus']==seller, 'Type'] = 2
                busdata.loc[busdata['Bus']==seller, 'PGi'] += (trading_units*1000*4)
                busdata_array = busdata.to_numpy()
                power_flow = PowerFlow(busdata_array, linedata) 
                V, Del, Iter, check = power_flow.loadflow(busdata_array[:, 2], busdata_array[:, 3], BMva)
                Lij,Sij,Pij,Qij,Pg,Qg, Pi, Qi, Iij, Pl = power_flow.powerflow(V, np.deg2rad(Del), BMva)
                
                
                for i in range(10):
                    for j in range(10):
                        if((i<j) and (limits[i][j] - max(np.abs(Pij[i][j]),np.abs(Pij[j][i])) < 0) or (check == False)):
                            rejected.append((buyer,seller))
                            utility_matrix.loc[seller, buyer] = 0
                            busdata.loc[busdata['Bus']==seller, 'PGi'] -= (trading_units*1000*4)
                            if busdata.loc[busdata['Bus'] == seller, 'PGi'].values[0] == 0:
                                busdata.loc[busdata['Bus'] == seller, 'Type'] = 3

                            flag = True
                            print(f"Period: {period}, Rejecting Transaction: 1, {seller}, Line Limit: {limits[i][j]}, Line: {i} -> {j}, Current Power Flow ({i}->{j}): {np.abs(Pij[i][j])}  Total Utility {max_utility},  Trading Units {trading_units}")
                            break
                        
                    if flag:
                        break
                     
                if flag:
                    continue
                      
                
                data.append([period, V, Del, Lij, Sij, Pij, Qij, Pg, Qg, Pi, Qi, Iij, Pl])
                total_utilities_list.append(create_transaction_df(period, seller, 1, mini_price, mini_price,
                                                              trading_units, 0, 0, 0,
                                                              0, 0))
                print(f"Transaction: Period {period},Buyer: 1, Seller: {seller}, Total Utility {max_utility}, Trading Units {trading_units}, Price{mini_price}")
                
                
                
                
        #GRID TO DEMAND
        buyer_data = period_data[period_data['Category'] == 'B']
        for _, row_buyer in buyer_data.iterrows():
            
            if round(row_buyer['Value'],3)>0:
                buyer = row_buyer['Prosumer_Id']
                trading_units = round(row_buyer['Value'],3)
                UBP = row_buyer['Unit Price']
                total_utilities_list.append(create_transaction_df(period, 1, buyer, UBP, UBP,
                                                              trading_units, 0, 0, 0,
                                                              0, 0))
            
    total_utilities_df = pd.concat(total_utilities_list, ignore_index=True)
    return total_utilities_df


start_time = time.time()
result_transactions = make_transactions_with_priority(df, ratings_df, sdr_df)
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

# GRID power flow increase - 1000 watts, 1200 - limit
# trading power flow decrease




C:\Users\sdash\AppData\Local\Temp\ipykernel_11904\70086540.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_utilities_df = pd.concat([total_utilities_df, new_row], ignore_index=True)


C:\Users\sdash\AppData\Local\Temp\ipykernel_11904\70086540.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_utilities_df = pd.concat([total_utilities_df, new_row], ignore_index=True)


Period: 1, Transaction Approved: 4, 8
Transaction: Period 1, Buyer: 4, Seller: 8, Total Utility 5.124612, Trading Units 0.084, Price 7.86
Period: 1, Transaction Approved: 4, 5
Transaction: Period 1, Buyer: 4, Seller: 5, Total Utility 3.974278, Trading Units 0.056, Price 6.26
Period: 1, Transaction Approved: 4, 2
Transaction: Period 1, Buyer: 4, Seller: 2, Total Utility 3.927316, Trading Units 0.324, Price 6.52
Period: 1, Transaction Approved: 4, 10
Transaction: Period 1, Buyer: 4, Seller: 10, Total Utility 3.903552, Trading Units 0.013, Price 6.47
Period: 1, Transaction Approved: 4, 9
Transaction: Period 1, Buyer: 4, Seller: 9, Total Utility 3.74319, Trading Units 0.385, Price 6.96
Period: 1, Transaction Approved: 7, 9
Transaction: Period 1, Buyer: 7, Seller: 9, Total Utility 2.586218, Trading Units 0.151, Price 7.07
Transaction: Period 1,Buyer: 1, Seller: 6, Total Utility 0, Trading Units 0.086, Price 5.73


C:\Users\sdash\AppData\Local\Temp\ipykernel_11904\70086540.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_utilities_df = pd.concat([total_utilities_df, new_row], ignore_index=True)
C:\Users\sdash\AppData\Local\Temp\ipykernel_11904\70086540.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_utilities_df = pd.concat([total_utilities_df, new_row], ignore_index=True)


Period: 2, Transaction Approved: 10, 8
Transaction: Period 2, Buyer: 10, Seller: 8, Total Utility 3.453177, Trading Units 0.056, Price 6.68
Period: 2, Transaction Approved: 4, 5
Transaction: Period 2, Buyer: 4, Seller: 5, Total Utility 1.76514, Trading Units 0.214, Price 4.4
Period: 2, Transaction Approved: 4, 2
Transaction: Period 2, Buyer: 4, Seller: 2, Total Utility 1.577723, Trading Units 0.073, Price 4.83
Period: 2, Transaction Approved: 6, 2
Transaction: Period 2, Buyer: 6, Seller: 2, Total Utility 1.567908, Trading Units 0.139, Price 8.5
Period: 2, Transaction Approved: 6, 9
Transaction: Period 2, Buyer: 6, Seller: 9, Total Utility 1.53573, Trading Units 0.164, Price 8.44
Period: 2, Transaction Approved: 6, 8
Transaction: Period 2, Buyer: 6, Seller: 8, Total Utility 1.500687, Trading Units 0.059, Price 8.81


...

Period: 30, Transaction Approved: 6, 9
Transaction: Period 30, Buyer: 6, Seller: 9, Total Utility 4.994828, Trading Units 0.046, Price 5.95
Period: 30, Transaction Approved: 4, 10
Transaction: Period 30, Buyer: 4, Seller: 10, Total Utility 4.667707, Trading Units 0.311, Price 6.57
Period: 30, Transaction Approved: 4, 5
Transaction: Period 30, Buyer: 4, Seller: 5, Total Utility 3.880363, Trading Units 0.003, Price 5.05
Period: 30, Transaction Approved: 4, 8
Transaction: Period 30, Buyer: 4, Seller: 8, Total Utility 3.807606, Trading Units 0.202, Price 6.09
Period: 30, Transaction Approved: 7, 9
Transaction: Period 30, Buyer: 7, Seller: 9, Total Utility 3.387567, Trading Units 0.401, Price 5.68
Period: 30, Transaction Approved: 3, 2
Transaction: Period 30, Buyer: 3, Seller: 2, Total Utility 3.107731, Trading Units 0.108, Price 6.29
Period: 30, Transaction Approved: 7, 8
Transaction: Period 30, Buyer: 7, Seller: 8, Total Utility 2.704368, Trading Units 0.045, Price 5.35
Transaction: Perio

C:\Users\sdash\AppData\Local\Temp\ipykernel_11904\70086540.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_utilities_df = pd.concat([total_utilities_df, new_row], ignore_index=True)
C:\Users\sdash\AppData\Local\Temp\ipykernel_11904\70086540.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_utilities_df = pd.concat([total_utilities_df, new_row], ignore_index=True)


Period: 31, Transaction Approved: 4, 9
Transaction: Period 31, Buyer: 4, Seller: 9, Total Utility 5.09168, Trading Units 0.596, Price 6.11
Period: 31, Rejecting Transaction: 2, 9, Line Limit: 2500, Line: 6 -> 8, Current Power Flow (6->8): 3029.7857419396305,  Total Utility 4.912336,  Trading Units 0.164 
Period: 31, Rejecting Transaction: 3, 9, Line Limit: 2500, Line: 6 -> 8, Current Power Flow (6->8): 3196.4567693537406,  Total Utility 3.029532,  Trading Units 0.206 
Period: 31, Transaction Approved: 2, 5
Transaction: Period 31, Buyer: 2, Seller: 5, Total Utility 2.991691, Trading Units 0.164, Price 5.88
Period: 31, Rejecting Transaction: 6, 9, Line Limit: 2500, Line: 6 -> 8, Current Power Flow (6->8): 2535.4621145405545,  Total Utility 2.785077,  Trading Units 0.04 
Period: 31, Rejecting Transaction: 7, 9, Line Limit: 2500, Line: 6 -> 8, Current Power Flow (6->8): 3193.833453326219,  Total Utility 1.720992,  Trading Units 0.206 
Period: 31, Transaction Approved: 3, 8
Transaction: Per

C:\Users\sdash\AppData\Local\Temp\ipykernel_11904\70086540.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_utilities_df = pd.concat([total_utilities_df, new_row], ignore_index=True)
C:\Users\sdash\AppData\Local\Temp\ipykernel_11904\70086540.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_utilities_df = pd.concat([total_utilities_df, new_row], ignore_index=True)


Period: 32, Transaction Approved: 4, 5
Transaction: Period 32, Buyer: 4, Seller: 5, Total Utility 2.160477, Trading Units 0.09, Price 5.57
Period: 32, Transaction Approved: 4, 2
Transaction: Period 32, Buyer: 4, Seller: 2, Total Utility 1.58032, Trading Units 0.061, Price 3.92
Period: 32, Transaction Approved: 4, 10
Transaction: Period 32, Buyer: 4, Seller: 10, Total Utility 1.503194, Trading Units 0.02, Price 4.11
Period: 32, Transaction Approved: 4, 8
Transaction: Period 32, Buyer: 4, Seller: 8, Total Utility 1.325606, Trading Units 0.007, Price 3.9
Period: 32, Transaction Approved: 4, 3
Transaction: Period 32, Buyer: 4, Seller: 3, Total Utility 1.308411, Trading Units 0.059, Price 4.51
Period: 32, Transaction Approved: 7, 9
Transaction: Period 32, Buyer: 7, Seller: 9, Total Utility 0.702768, Trading Units 0.371, Price 4.92
Elapsed time: 27.832995653152466 seconds


In [20]:
result_transactions.to_csv('result_transactions.csv', index=False)
result_transactions


,Period,Seller,Buyer,SDR Price,Trading Price,Trading Energy,Seller Profit,Buyer Profit,Seller Utility,Buyer Utility,Total_Utility
0,1,8,4,7.62,7.86,0.084,0.358,0.180,3.356,1.527,5.124612
1,1,5,4,7.62,6.26,0.056,0.149,0.209,1.379,2.882,3.974278
2,1,2,4,7.62,6.52,0.324,0.946,1.128,1.514,2.594,3.927316
3,1,10,4,7.62,6.47,0.013,0.037,0.046,1.506,2.592,3.903552
4,1,9,4,7.62,6.96,0.385,1.294,1.170,1.881,1.990,3.743190
...,...,...,...,...,...,...,...,...,...,...,...
252,32,3,4,5.56,4.51,0.059,0.054,0.147,0.633,2.067,1.308411
253,32,9,7,5.56,4.92,0.371,0.490,0.772,0.528,1.331,0.702768
254,32,1,4,7.00,7.00,0.102,0.000,0.000,0.000,0.000,0.000000
255,32,1,6,7.00,7.00,0.034,0.000,0.000,0.000,0.000,0.000000


In [21]:
result_transactions['Trading Energy'].sum()

57.003